In [ ]:
%cd ..
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp

# Load an example dataset and example kernels

In [ ]:
scale_factor = 4
resize_method = tf.image.ResizeMethod.BICUBIC

builder_kwargs = {
    'resize_method': resize_method,
    'scale': scale_factor,
    'antialias': True
}
images = tfds.load(name="set14", split="test", builder_kwargs=builder_kwargs) \
                .map(datasets.map_on_dict(datasets.to_float32)) \
                .map(datasets.map_on_dict(datasets.from_255_to_1_range))

In [ ]:
image_dict = datasets.get_one_example(images, index=2)
image_hr = image_dict['hr'][None, ...]
image_lr = image_dict['lr'][None, ...]

# Reconstruct the image

In [ ]:
# Load the denoiser
denoiser, (denoiser_min_stddev, denoiser_max_stddev) = dppp.load_denoiser('models/drugan+_0.0-0.2.h5')

# Define the noise stddev used for the stochastic evaluation of the prior
if denoiser_min_stddev <= 0.1 <= denoiser_max_stddev:
    denoiser_stddev = 0.1
else:
    denoiser_stddev = denoiser_max_stddev

# Callbacks: Print the PSNR every 20th step
callbacks = [ dppp.callback_print_psnr('psnr', 20, image_hr) ]

# Reconstruct the image
reconstructed = dppp.dmsp_super_resolve(image_lr, scale_factor, denoiser, denoiser_stddev,
                                        num_steps=300, callbacks=callbacks)

In [ ]:
# Print PSNR and FSIM
psnr = dppp.psnr(image_hr, reconstructed).numpy()[0]
fsim = dppp.fsim(image_hr, reconstructed).numpy()[0]
print(f"Reconstructed PSNR: {psnr:0.2f}, FSIM: {fsim:0.4f}")

# Visualize
visualize.draw_images([image_hr[0], image_lr[0], reconstructed[0]])